## Time-series forecasting of PV production

In [ ]:
import timeit
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
try:
    import seaborn as sns
    # Seaborn style (figure aesthetics only)
    sns.set(context='paper', style='whitegrid', font_scale=1.2)
    sns.set_style('ticks', {'xtick.direction':'in', 'ytick.direction':'in'})
except ImportError:
    print('Seaborn not installed. Going without it.')

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA

In [ ]:
from scipy import stats

### PV Data

5 seconds resolution MiRIS PV from 13/05/2019 to 21/06/2019.

In [ ]:
pv = pd.read_csv('miris_pv.csv', index_col=0, parse_dates=True)

In [ ]:
# Resampling the dataset from 5-seconds to 15-minutes resolution (using mean)
pv = pv.resample('15min').mean()

### Weather Data

15-minute resolution weather data

The file is composed of forecast of several weather variables:

    CD = low clouds (0 to 1)
    CM = medium clouds (0 to 1)
    CU = high clouds (0 to 1)
    PREC = precipitation (mm / 15 min)
    RH2m = relative humidity (%)
    SNOW = snow height (mm)
    ST = Surface Temperature (°C)
    SWD = Global Horizontal Irradiance (W/m2)
    SWDtop = Total Solar Irradiance at the top of the atmosphere (W/m2)
    TT2M = temperature 2 meters above the ground (°C)
    WS100m = Wind speed at 100m from the ground (m/s)
    WS10m = Wind speed at 10m from the ground (m/s)

In [ ]:
we = pd.read_csv('weather_data.csv', index_col=0, parse_dates=True)

### Cleaning data

In [ ]:
# Dropping SNOW and SWDtop from the dataset
we.drop('SNOW', axis=1, inplace=True)
we.drop('SWDtop', axis=1, inplace=True)

In [ ]:
# Joining pv production and weather data into single dataframe
df = pd.concat([pv, we], axis=1)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

In [ ]:
df[['PV']].plot(figsize=(12,4.5))
plt.show()

### Features engineering from time-series data

In [ ]:
def engineer_features(dataframe, window=24, steps_ahead=1, 
                      copy_data=True, resample=True, drop_nan_rows=True):
    """ Engineer features from time-series data

    Features engineering from the time-series data by using time-shift,
    first-differencing, rolling window statistics, cyclical transforms,
    and encoding. NaN values are dropped from the final dataset.

    Arguments
    ---------
    dataframe: pd.DataFrame
        Pandas dataframe holding the original time-series data 
        (in the long table format).
    window: int
        Window size for the past observations (for time-shifting).
    steps_ahead: int
        Number of time steps ahead for multi-step forecasting 
        (steps_ahead=1 means single-step ahead forecasting).
    copy_data: bool
        True/False indicator for making a local copy of the dataframe
        inside the function.
    resample: bool
        True/False indicator for resampling data to hourly frequency.
   drop_nan_rows: bool
        True/False indicator to drop rows with NaN values.

    Returns
    -------
    df: pd.DataFrame
        Pandas dataframe with newly engineered features (long format).
    
    Raises
    ------
    Nothing.
    """
    
    if copy_data:
        df = dataframe.copy()
    else:
        df = dataframe
    if resample:
        df = df.resample('1H').mean()
    
    # Engineer features from time-series data
    columns = df.columns
    for col in columns:
        for i in range(1, window+1):
            # Shift data by lag of 1 to window=24 hours
            df[col+'_{:d}h'.format(i)] = df[col].shift(periods=i)  # time-lag
    for col in columns:
        df[col+'_diff'] = df[col].diff()  # first-difference
    df['PV_diff24'] = df['PV'].diff(24)

    # Rolling windows (24-hours) on time-shifted PV production
    df['roll_mean'] = df['PV_1h'].rolling(window=24, win_type='hamming').mean()
    df['roll_max'] = df['PV_1h'].rolling(window=24).max()
    
    # Hour-of-day indicators with cyclical transform
    dayhour_ind = df.index.hour
    df['hr_sin'] = np.sin(dayhour_ind*(2.*np.pi/24))
    df['hr_cos'] = np.cos(dayhour_ind*(2.*np.pi/24))
    
    # Month indicators with cyclical transform
    month_ind = df.index.month
    df['mnth_sin'] = np.sin((month_ind-1)*(2.*np.pi/12))
    df['mnth_cos'] = np.cos((month_ind-1)*(2.*np.pi/12))

    # Encoding sunshine hours
    sun_ind = df['PV'] > 0.
    df['sun'] = sun_ind.astype(int)

    # Forecast horizont
    if steps_ahead == 1:
        # Single-step ahead forecasting
        df['PV+0h'] = df['PV'].values
    else:
        # Multi-step ahead forecasting
        for i in range(steps_ahead):
            df['PV'+'+{:d}h'.format(i)] = df['PV'].shift(-i)
    del df['PV']

    # Drop rows with NaN values
    if drop_nan_rows:
        df.dropna(inplace=True)

    return df

In [ ]:
# Single-step model
df2 = engineer_features(df)
df2.head()

### Train, validation, and test datasets (time-series data)

In [ ]:
def prepare_data(dataframe, weather_forecast=False, copy_data=True):
    if copy_data:
        df = dataframe.copy()
    else:
        df = dataframe
    if weather_forecast is False:
        # Hour-ahead weather forecast is NOT being utilized
        df.drop(columns=['CD', 'CM', 'CU', 'PREC', 'RH2m', 'ST', 
                        'SWD', 'TT2M', 'WS100m', 'WS10m'],
                inplace=True)

    columns = df.columns.values
    outputs = [col_name for col_name in columns if 'PV+' in col_name]
    inputs = [col_name for col_name in columns if col_name not in outputs]
    # inputs (features)
    X = df[inputs]
    # outputs
    y = df[outputs]
    return X, y

In [ ]:
# Hour-ahead weather forecast is NOT being utilized
weather_forecast = False
# Prepare dataframe for a split into train, test sets
X, y = prepare_data(df2)

In [ ]:
# Train and test dataset split (w/o shuffle)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)

In [ ]:
# Print train and test set shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

### Single-step model pipeline with features selection

In [ ]:
model = 'SVR'  # 'RandomForest'

In [ ]:
if model == 'RandomForest':
    # Pipeline: SelectKBest and RandomForest
    # SelectKBest is used for features selection/reduction
    selectbest = SelectKBest(score_func=mutual_info_regression, k='all')
    forest = RandomForestRegressor(criterion='mse', bootstrap=True)
    # Creating a pipeline
    pipe = Pipeline(steps=[('preprocess', 'passthrough'), 
                           ('kbest', selectbest), 
                           ('forest', forest)])
    # Parameters of pipeline for the randomized search with cross-validation
    param_dists = {'preprocess': [None, StandardScaler()], 
                   'kbest__k': stats.randint(low=32, high=128), 
                   'forest__n_estimators': stats.randint(low=200, high=1000),
                   'forest__max_depth': [1, 3, 5, None], 
                   'forest__max_samples': stats.uniform(loc=0.2, scale=0.8),
                  }
elif model == 'SVR':
    # Pipeline: SelectKBest and SVR
    # SelectKBest is used for features selection/reduction
    selectbest = SelectKBest(score_func=mutual_info_regression, k='all')
    # Support Vector Regression 
    svr = SVR(kernel='rbf', gamma='scale')
    # Creating a pipeline
    pipe = Pipeline(steps=[('preprocess', 'passthrough'), 
                           ('kbest', selectbest), 
                           ('svr', svr)])
    # Parameters of pipeline for the randomized search with cross-validation
    param_dists = {'preprocess': [None, StandardScaler()], 
                   'kbest__k': stats.randint(low=32, high=128), 
                   'svr__C': stats.loguniform(1e0, 1e3),
                   'svr__epsilon': stats.loguniform(1e-5, 1e-2),
                  }
else:
    raise NotImplementedError('Model name "{}" is not recognized or implemented!'.format(model))

NITER = 100  # number of random search iterations
time_start = timeit.default_timer()
search = RandomizedSearchCV(estimator=pipe, param_distributions=param_dists, 
                            cv=TimeSeriesSplit(n_splits=3),
                            scoring='neg_mean_squared_error',
                            n_iter=NITER, refit=True, n_jobs=-1)
search.fit(X_train, y_train)
time_end = timeit.default_timer()
time_elapsed = time_end - time_start
print('Execution time (hour:min:sec): {}'.format(str(dt.timedelta(seconds=time_elapsed))))
print('Best parameter (CV score = {:.3f}):'.format(search.best_score_))
print(search.best_params_)

#### Feature importance analysis

In [ ]:
if model == 'RandomForest':
    # Feature importance analysis with random forests
    best_params = {'n_estimators': search.best_params_['forest__n_estimators'],
                   'max_depth': search.best_params_['forest__max_depth'],
                   'max_samples': search.best_params_['forest__max_samples'],
                  }
    forest = RandomForestRegressor(criterion='mse', **best_params)
    forest.fit(X_train, y_train)

    TOP = 15
    feature_importance = forest.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)[-TOP:]
    pos = np.arange(sorted_idx.shape[0]) + .25

    # Plot relative feature importance
    fig, ax = plt.subplots(figsize=(7,5))
    ax.barh(pos, feature_importance[sorted_idx][-TOP:], 
            align='center', color='magenta', alpha=0.6)
    plt.yticks(pos, X_train.columns[sorted_idx][-TOP:])
    ax.set_xlabel('Feature Relative Importance')
    ax.grid(axis='x')
    plt.tight_layout()
    plt.show()

### Single-step ahead prediction

In [ ]:
X_test.head()

In [ ]:
# Make single-step predictions for 24 hours ahead
y_preds = search.predict(X_test.values[:24,:])

In [ ]:
mse = mean_squared_error(y_test.values[:24], y_preds)
print('MSE:', mse.round(5))
mae = mean_absolute_error(y_test.values[:24], y_preds)
print('MAE:', mae.round(5))

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(y_test.index[:24], y_test.values[0:24], lw=2, label='true values')
plt.plot(y_test.index[:24], y_preds, ls='--', lw=1.5, marker='+', ms=10, label='predictions')
plt.text(y_test.index[20], 0.35, 'MAE: {:.3f}'.format(mae), horizontalalignment='center', fontweight='bold')
plt.legend(loc='upper right')
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.xlabel('Day/Hour')
plt.ylabel('PV power')
plt.show()

### Walk-forward multi-step prediction with a single-step model

In [ ]:
if weather_forecast:
    raise NotImplementedError('Walk forward is not implemented with hour-ahead weather forecast.')

In [ ]:
WALK = 12  # walk-forward for WALK hours
STEP = 24  # multi-step predict for STEP hours ahead

In [ ]:
# With STEP=24 and WALK=12, we are making a 24-hour ahead predictions 
# after each hour, and move forward in time for 12 hours in total. 
# In other words, we walk forward for 12 hours, and each time we move 
# forward (by one hour) we make a brand new 24-hour ahead predictions. 
# Predicted values are being utilized as past observations for making
# new predictions as we walk forward in time. Hence, as we move away in 
# time from the present moment we are relying more and more on predicted 
# values to make new predictions!

def walk_forward(X_values, y_predicted, window=24):
    """ Walk forward

    Preparing input matrix X for walk-forward single-step predictions.
    There are eleven different original variables (PV plus 10 weather 
    vars.), which have been time-shifted using the "window" method.
    NOTE: This function uses certain hard-coded elements adjusted for 
    the particular problem at hand.

    Arguments
    ---------
    X_values: np.array
        Input values for making predictions.
    Y_predicted: float
        Predicted value.
    window: int
        Window size of the past observations. It should be equal to 
        the window size that was used for features engineering.

    Returns
    -------
    X_values: np.array
        Input values time-shifted by a single time step, using the 
        walk forward approach.

    Raises
    ------
    Nothing.
    """

    # There are eleven different original
    # variables (PV plus 10 weather vars)
    X_parts = []
    j = 0; k = 0
    for i in range(11):
        k = j + window
        X_part = X_values[j:k]
        X_part = pd.Series(X_part)
        if i == 0:
            # time-shifted PV production
            X_part = X_part.shift(periods=1, fill_value=y_predicted)
        else:
            # time-shifted weather features
            X_part = X_part.shift(periods=1, fill_value=np.NaN)
            X_part.fillna(method='bfill', inplace=True)  # back-fill
        X_parts.append(X_part.values)
        j += window
    X_parts = np.asarray(X_parts).reshape(1,-1).flatten()
    X_rest = X_values[-19:]   # other features (hard-coded)
    # Update feature PV_diff with y_predicted
    X_rest[0] = X_parts[0] - X_parts[1]
    # Concatenate
    X_values = np.r_[X_parts, X_rest]
    return X_values

In [ ]:
def plot_predictions(walk, y_test, y_pred):
    plt.figure(figsize=(6,4))
    plt.title('walk forward +{:2d} hours'.format(walk+1))
    plt.plot(y_test.values[walk:walk+STEP], lw=2.5, label='true values')
    plt.plot(y_pred, ls='--', lw=1.5, marker='+', ms=10, label='predictions')
    mae = mean_absolute_error(y_test.values[walk:walk+STEP], y_pred)
    plt.text(STEP-2, 0.35, 'MAE: {:.3f}'.format(mae), 
             horizontalalignment='right', 
             fontweight='bold')
    plt.legend(loc='upper right')
    plt.ylim(top=0.5)
    plt.grid(axis='y')
    plt.xlabel('Hour')
    plt.ylabel('PV power')
    plt.show()

In [ ]:
# Do walk-forward predictions
for k in range(WALK):
    X_test_values = X_test.values[k,:]
    y_pred_values = []
    for i in range(STEP):
        # Predict next time-step value
        y_predict = search.predict(X_test_values.reshape(1,-1))[0]
        y_pred_values.append(y_predict)
        # Walk-forward for a single time step
        X_test_values = walk_forward(X_test_values, y_predict)
    # Plot walk-forward predictions against true values
    plot_predictions(k, y_test, y_pred_values)

### Multi-step model pipeline without features selection

In [ ]:
# Multi-step model (24-hours ahead)
df2 = engineer_features(df, steps_ahead=STEP)
# Prepare dataframe for a split into train, test sets
X, y = prepare_data(df2)

In [ ]:
# Train and test dataset split (w/o shuffle)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)

In [ ]:
# Print train and test set shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
X_test.head()

In [ ]:
multi_model = 'SVR'

In [ ]:
if multi_model == 'RandomForest':
    # Random Forest Regression (supports multi-output natively)
    forest = RandomForestRegressor(criterion='mse', bootstrap=True)
    # Creating a pipeline
    pipe = Pipeline(steps=[('preprocess', 'passthrough'), 
                           ('forest', forest)])
    # Parameters of pipeline for the randomized search with cross-validation
    param_dists = {'preprocess': [None, StandardScaler()], 
                   'forest__n_estimators': stats.randint(low=200, high=1000),
                   'forest__max_depth': [1, 3, 5, None], 
                   'forest__max_samples': stats.uniform(loc=0.2, scale=0.8),
                  }
elif multi_model == 'SVR':
    # Support Vector Regression (does NOT support multi-output natively)
    svr = MultiOutputRegressor(SVR(kernel='rbf', gamma='scale'))
    # Creating a pipeline
    pipe = Pipeline(steps=[('preprocess', 'passthrough'), 
                           ('svr', svr)])
    # Parameters of pipeline for the randomized search with cross-validation
    param_dists = {'preprocess': [None, StandardScaler()], 
                   'svr__estimator__C': stats.loguniform(1e0, 1e3),
                   'svr__estimator__epsilon': stats.loguniform(1e-5, 1e-2),
                  }
elif multi_model == 'PCA+SVR':
    # Principal Component Analysis (PCA) is used for decomposing 
    # (i.e. projecting) features into the lower-dimensional space
    # while retaining maximum amount of the variance.
    pca = PCA(whiten=True, svd_solver='full')
    # Support Vector Regression (does NOT support multi-output natively)
    svr = MultiOutputRegressor(SVR(kernel='rbf', gamma='scale'))
    # Creating a pipeline
    pipe = Pipeline(steps=[('pca', pca), 
                           ('svr', svr)])
    # Parameters of pipeline for the randomized search with cross-validation
    param_dists = {'pca__n_components': stats.uniform(),
                   'svr__estimator__C': stats.loguniform(1e0, 1e3),
                   'svr__estimator__epsilon': stats.loguniform(1e-5, 1e-2),
                  }
else:
    raise NotImplementedError('Model name "{}" is not recognized or implemented!'.format(model))

NITER = 100  # number of random search iterations
time_start = timeit.default_timer()
search_multi = RandomizedSearchCV(estimator=pipe, param_distributions=param_dists, 
                                  cv=TimeSeriesSplit(n_splits=3),
                                  scoring='neg_mean_squared_error',
                                  n_iter=NITER, refit=True, n_jobs=-1)
search_multi.fit(X_train, y_train)
time_end = timeit.default_timer()
time_elapsed = time_end - time_start
print('Execution time (hour:min:sec): {}'.format(str(dt.timedelta(seconds=time_elapsed))))
print('Best parameter (CV score = {:.3f}):'.format(search_multi.best_score_))
print(search_multi.best_params_)

In [ ]:
def plot_multi_step_predictions(walk, y_test, y_pred):
    plt.figure(figsize=(6,4))
    plt.title('walk forward +{:2d} hours'.format(walk+1))
    plt.plot(y_test, lw=2.5, label='true values')
    plt.plot(y_pred, ls='--', lw=1.5, marker='+', ms=10, label='predictions')
    mae = mean_absolute_error(y_test, y_pred)
    plt.text(STEP-2, 0.35, 'MAE: {:.3f}'.format(mae), 
             horizontalalignment='right', 
             fontweight='bold')
    plt.legend(loc='upper right')
    plt.ylim(top=0.5)
    plt.grid(axis='y')
    plt.xlabel('Hour')
    plt.ylabel('PV power')
    plt.show()

In [ ]:
# Do multi-step ahead predictions
for k in range(WALK):
    X_test_values = X_test.values[k+20,:]  # +20 hard-coded shift to align views with that
    y_test_values = y_test.values[k+20,:]  # of walk-forward predictions for easy comparison
    y_predict = search_multi.predict(X_test_values.reshape(1,-1)).flatten()
    # Plot multi-step predictions against true values
    plot_multi_step_predictions(k, y_test_values, y_predict)